# Cross validated DVs



In [ ]:
import sys, re
sys.path.insert(1, '../../../figure_code/')
from my_imports import *
import pingouin as pg
from tqdm import tqdm
import random
from helpers.phys_helpers import get_sortCells_for_rat, equalize_neurons_across_regions, datetime, savethisfig, xcorr
from helpers.rasters_and_psths import get_neural_activity
from helpers.physdata_preprocessing import load_phys_data_from_Cell
from scipy import stats
from DVcc_sims import crosscorrelate_DVs, plot_cc

def get_filenames(decoding_dir):
    filenames = sorted([fn for fn in os.listdir(decoding_dir) if re.findall(".npy", fn)])
    param_file = [fn for fn in filenames if 'params' in fn][0]  
    p = np.load(decoding_dir + param_file, allow_pickle = True).item()
    filenames.remove(param_file)
    return p, filenames


CH_DECODING_DIR = 'DV_decoding/'
SAVEDIR = SPEC.RESULTDIR + 'DV_decoding/'

this_dir = os.path.join(SPEC.RESULTDIR, CH_DECODING_DIR)
p, filenames = get_filenames(this_dir)

p['binsize'] = 5  # in ms, to get at latencies
p['filter_w'] = 50  # in ms, still need to smooth a bit
p['filter_type'] = 'gaussian'
p['end_time'] = [800]


DV_summary = dict()
for var in ['DV_cc', 'DV_peak', 'DV_cc_shuff', 'lags']:
    DV_summary[var] = []
    
align_id = 0
align_to = p['align_to'][align_id]
align_name = p['align_name'][align_id]

In [ ]:
for filenum, fn in enumerate(filenames):

    summary = np.load(this_dir + fn, allow_pickle=True).item()
    rat = summary['prm']['filename'][:4]

    print(rat)
    _, this_datadir = get_sortCells_for_rat(rat, SPEC.DATADIR)


    df_trial, df_cell, _ = load_phys_data_from_Cell(
    this_datadir + os.sep + summary['prm']['filename'])

    summary = summary['clickson_masked']

    df_trial = df_trial[df_trial['stim_dur_s_actual']
                    >= p['stim_thresh']].reset_index(drop=True)

    # equalize neurons across regions
    df_cell = df_cell[df_cell['stim_fr'] >= p['fr_thresh']].reset_index(drop=True)
    df_cell = equalize_neurons_across_regions(df_cell, p['regions'])

    # equalize left right trials
    random.seed(10)
    n = df_trial['pokedR'].value_counts().min()
    idx_R = random.sample(list(np.where(df_trial['pokedR'] == 1)[0]), n)[:n]
    idx_L = random.sample(list(np.where(df_trial['pokedR'] == 0)[0]), n)[:n]
    df_trial = df_trial.loc[np.sort(idx_R + idx_L)].reset_index(drop=True)
    ntrials = len(df_trial)
    print("trial_count: {}".format(ntrials))

    X = dict()
    X = dict()
    for reg in p['regions']:
        X[reg], ntpts_per_trial = get_neural_activity(
            df_cell, df_trial, reg, p, align_id)
        
    idx_trial = np.concatenate(([0], np.cumsum(ntpts_per_trial)))
    max_tpts = max(ntpts_per_trial)
    num_trials = len(ntpts_per_trial)
    
    DV = dict()
    for reg in p['regions']:
        DV[reg] = np.nan * np.zeros((num_trials, max_tpts))
        cv_dict = summary[reg]['cv_summary'][0]
        for fold in range(p['nfolds']):
            for tidx in cv_dict['test_trials'][fold]:
                mdl_coefs = cv_dict['mdl_coefs'][fold]
                intercept = cv_dict['mdl_intercept'][fold]
                this_slice = slice(idx_trial[tidx], idx_trial[tidx + 1])
                test_index = np.arange(this_slice.start, this_slice.stop)
                DV[reg][tidx, 0:ntpts_per_trial[tidx]] = mdl_coefs @ X[reg][:,test_index] + intercept
            
    # cross correlate DVs
    DV_cc = crosscorrelate_DVs(DV, p, shuffle = False)
    DV_cc_shuff = crosscorrelate_DVs(DV, p, shuffle = True)

    
    
    DV_summary['DV_cc'].append(DV_cc['mean'])
    DV_summary['DV_peak'].append(DV_cc['peak'])
    DV_summary['DV_cc_shuff'].append(DV_cc_shuff['mean'])
    DV_summary['lags'].append(DV_cc['lags'])

In [ ]:
# now plot cross-correlation 
fig = plt.figure(figsize = (6,5))
sns.set_context("talk")

ax = fig.gca()

compute_shuffle = True

def get_DV_cc(DV_summary, compute_shuffle):
    if compute_shuffle:
        var = 'DV_cc_shuff'
    else:
        var = 'DV_cc'
    # i should make sure all the lags are the same!!
    DV_cc = dict()
    DV_cc['lags'] = DV_summary['lags'][0]
    DV_cc['mean'] = np.nanmean(DV_summary[var], axis = 0)
    DV_cc['sem'] = np.nanstd(DV_summary[var], axis = 0)/np.sqrt(len(DV_summary[var]))
    peak_idx = np.argmax(DV_cc['mean'])
    DV_cc['peak'] = DV_cc['lags'][peak_idx] 
    
    return DV_cc

plot_cc(get_DV_cc(DV_summary, False), 'k', ax, 'Mean across rats', alpha = 0.5)
ax.axvline(DV_cc['peak'], c = 'k', ls = '--')
plot_cc(get_DV_cc(DV_summary, True), [0.5, 0.5, 0.5], ax, 'shuffle', alpha = 0.2)
ax.legend(frameon = False)
ax.axvline(0, c = 'k', ls = ':')
ax.axhline(0, c = 'k',ls = ':')
ax.set_xlim([-0.4, 0.4])
ax.set_xlabel('Lags [s]')
ax.set_ylabel('CV Decision variable \n correlation (ADS, FOF) $\pm$ SEM')
ax.set_ylim([-0.1, 1.2])
for r, reg in enumerate(np.flip(p['regions'])):
    ax.text(-0.1 + r*0.2,  
        ax.get_ylim()[1], 
        reg + ' leads', 
        fontsize=16, 
        va='center', 
        ha='center', 
        backgroundcolor='w')
sns.despine()

peaks = DV_summary['DV_peak']
# Perform one-sample t-test
t_statistic, p_value = stats.ttest_1samp(peaks, 0)

# Print results
print("T-statistic:", t_statistic)
print("P-value:", p_value)
print("Mean peak: {}".format(np.mean(peaks)))
savethisfig(SPEC.FIGUREDIR, 'figure2/figure2_DV_crosscorrelation')


output_file = open(SPEC.RESULTDIR + "figure2_DV_summary.txt",'w')
print("\n\nT-test for to check if peak is different from 0 for %s sessions" % len(peaks), file = output_file)
print("\nMean peak: {}".format(np.mean(peaks)), file= output_file)
print("\nsem: {}".format(np.std(peaks)/np.sqrt(len(peaks))), file= output_file)
print("\nT-statistic: {}".format(t_statistic), file= output_file)
print("\nP-value: {}".format(p_value), file= output_file)

# DVcc_sims plots

In [ ]:
from DVcc_sims import *
sns.set_context("talk")


In [ ]:

N_neurons = 20
N_trials = 200
ff_delays = 5

# for simulations
params = dict()
params['dt'] = 0.001
params['history_bias'] = True
params['bound'] = 8.
params['leak'] = 0.99
params['T'] = [0.2, 1.0]



# for decoding
p = dict()
p['regions'] = ['A', 'B']
p['cols'] = SPEC.COLS
p['fr_thresh'] = 1.0 # firing rate threshold for including neurons
p['stim_thresh'] = 0.0 # stimulus duration threshold for including trials
p['align_to'] = ['clicks_on']
p['align_name'] = ['clickson_masked']
p['pre_mask'] = [None]
p['post_mask'] = ['clicks_off']
p['start_time'] = [0,]
p['end_time'] = [1000] # in ms
p['binsize'] = 50 # in ms
p['filter_type'] = 'gaussian'
p['filter_w'] = 75 # in ms
p['Cs'] = np.logspace(-7,3,200)  # cross-validation parameter
p['nfolds'] = 10  # number of folds for cross-validation
p['n_repeats'] = 1 # number of repeats for cross-validation


# for DV cross correlations
prm = dict()
prm['regions'] = ['A', 'B']
prm['cols'] = SPEC.COLS
prm['align_to'] = ['clicks_on']
prm['align_name'] = ['clickson_masked']
prm['pre_mask'] = [None]
prm['post_mask'] = ['clicks_off']
prm['start_time'] = [0]
prm['end_time'] = [800] # in ms
prm['binsize'] = 5 # in ms
prm['filter_type'] = 'gaussian'
prm['filter_w'] = 50 # in ms




In [ ]:
# simulate data
pc_task = PoissonClicks(**params)
ns = NeuralSimulator(**params)
click_data, task_data = pc_task.get_trial_batch()
latents, spikes, spike_rate_dt, task_data = ns.simulate_trials(click_data, task_data)


In [ ]:
fig_dyn = plt.figure(constrained_layout = True, figsize = (12,8))
gs = fig_dyn.add_gridspec(5,4, width_ratios=[10,10,5,1])

xlabel_dyn = ns.get_latents_labels()

# dynamics matrix during accumulation
ax0 = fig_dyn.add_subplot(gs[0:2, 0])
im = ax0.matshow(ns.A[0], cmap = 'Blues', vmin = 0., vmax = 1.)
ax0.set_title('$A_{acc}$')
ax0.set_yticks(ticks = range(2*ns.N_latents_per_region), labels = xlabel_dyn)
ax0.set_xticks(ticks = range(2*ns.N_latents_per_region), labels = xlabel_dyn, rotation = 90)


# dynamics matrix during accumulation
ax0 = fig_dyn.add_subplot(gs[0:2, 1])
ax0.matshow(ns.A_delay[0], cmap = 'Blues', vmin = 0., vmax = 1.)
ax0.set_title('$A^{ff}_{acc}$')
ax0.set_yticks(ticks = range(2*ns.N_latents_per_region), labels = xlabel_dyn)
ax0.set_xticks(ticks = range(2*ns.N_latents_per_region), labels = xlabel_dyn, rotation = 90);

# dynamics matrix during accumulation
ax0 = fig_dyn.add_subplot(gs[0:2, 2])
ax0.matshow(ns.B[0], cmap = 'Blues', vmin = 0., vmax = 1.)
ax0.set_title('$B_{acc}$')
ax0.set_yticks(ticks = range(2*ns.N_latents_per_region), labels = xlabel_dyn)
ax0.set_xticks(ticks = [0,1], labels = { 'δR-δL', 'history'}, rotation = 45)

# add the colorbar at the top of the plot
ax0 = fig_dyn.add_subplot(gs[0:2, 3])
cbar = plt.colorbar(im, cax=ax0, orientation='vertical')


ax0 = fig_dyn.add_subplot(gs[3, :])
ax0.matshow(ns.C.T, vmin = -2, vmax = 2, cmap = 'RdBu', aspect = 'auto')
ax0.set_xlabel('Neuron number')
ax0.set_ylabel('Latents')
ax0.set_title('$C$')
ax0.set_yticks(ticks = range(2*ns.N_latents_per_region), labels = xlabel_dyn)


savethisfig(SPEC.FIGUREDIR + "figure2/", 'figure2_DVsweeps_params')


In [ ]:
sns.set_context("paper")

ns.plot_activity_portrait(task_data)
savethisfig(SPEC.FIGUREDIR + "figure2/", 'figure2_DVsweeps_activity')

In [ ]:
ns.clicks.shape[1]

#